In [1]:
import pandas as pd
import os
import numpy as np
import pickle

In [2]:
df1 = pd.read_csv('D:/programs_demo/T20_sim/extractedData/full_matches_data.csv', dtype = {'season' : str})

In [3]:
df = df1.drop(['season', 'start_date', 'batting_team', 'bowling_team', 'non_striker', 'byes', 'legbyes', 'penalty', 'other_wicket_type', 'other_player_dismissed', 'File_Name'], axis=1)

In [4]:
def zeroDiv(x, y):
    return round((y and x/y), 4)

In [5]:
def oneDiv(x, y):
    return (1 if y == 0 else round(x/y, 4))

In [6]:
overall = {}
balls = len(df)
wides = df['wides'].count()
Balls = balls - wides
outs = df['wicket_type'].count()
runs = df['runs_off_bat'].sum()
run = df['runs_off_bat'].value_counts()

overall['wkt'] = zeroDiv(outs, balls)
overall['wide'] = zeroDiv(wides, balls)
overall['noball'] = zeroDiv(df['noballs'].count(), balls)
overall['0'] = zeroDiv(run[0], Balls)
overall['1'] = zeroDiv(run[1], Balls)
overall['2'] = zeroDiv(run[2], Balls)
overall['3'] = zeroDiv(run[3], Balls)
overall['4'] = zeroDiv(run[4], Balls)
overall['6'] = zeroDiv(run[6], Balls)
overall['runs'] = runs
overall['sr'] = zeroDiv(runs, Balls)
overall['Balls'] = Balls
overall['outs'] = outs
overall['balls'] = balls
overall['wkt_type'] = dict(round(df['wicket_type'].value_counts()/df['wicket_type'].count(),4))

In [10]:
file = open('extractedData/overall', 'wb')
pickle.dump(overall, file)
file.close()

In [11]:
phase = {}
phs = ['pp', 'mid', 'death']
st = [0, 6, 16]
en = [6, 16, 20]
for i in range(3):
    phase[phs[i]] = {}
    df1 = df.loc[(df['ball'] > st[i]) & (df['ball'] < en[i])]
    balls = len(df1)
    wides = df1['wides'].count()
    Balls = balls - wides
    outs = df1['wicket_type'].count()
    runs = df1['runs_off_bat'].sum()
    run = df1['runs_off_bat'].value_counts()

    phase[phs[i]]['wkt'] = zeroDiv(outs, balls)
    phase[phs[i]]['wide'] = zeroDiv(wides, balls)
    phase[phs[i]]['noball'] = zeroDiv(df1['noballs'].count(), balls)
    phase[phs[i]]['0'] = zeroDiv(run[0], Balls)
    phase[phs[i]]['1'] = zeroDiv(run[1], Balls)
    phase[phs[i]]['2'] = zeroDiv(run[2], Balls)
    phase[phs[i]]['3'] = zeroDiv(run[3], Balls)
    phase[phs[i]]['4'] = zeroDiv(run[4], Balls)
    phase[phs[i]]['6'] = zeroDiv(run[6], Balls)
    phase[phs[i]]['runs'] = runs
    phase[phs[i]]['sr'] = zeroDiv(runs, Balls)
    phase[phs[i]]['Balls'] = Balls
    phase[phs[i]]['outs'] = outs
    phase[phs[i]]['balls'] = balls
    phase[phs[i]]['wkt_type'] = dict(round(df1['wicket_type'].value_counts()/df1['wicket_type'].count(),4))

In [12]:
file = open('D:/programs_demo/T20_sim/extractedData/phaseOverallData', 'wb')
pickle.dump(phase, file)
file.close()

In [10]:
def xDiv(x, y):
    return round((y and x/y or x), 4)

In [90]:
batStats = pd.DataFrame(columns = ['Name', 'pp_runs', 'pp_avg', 'pp_sr', 'mid_runs', 'mid_avg', 'mid_sr', 'death_runs', 'death_avg', 'death_sr'])

allPlayersBat = {}

phs = ['pp', 'mid', 'death']
st = [0, 6, 16]
en = [6, 16, 20]
batStats["Name"] = df['striker'].unique()

for name in df['striker'].unique():
    allPlayersBat[name] = {}
    df2 = df.loc[df['striker'] == name]
    
    for i in range(3):
        allPlayersBat[name][phs[i]] = {}
        df1 = df2.loc[(df2['ball'] > st[i]) & (df2['ball'] < en[i])]
        balls = len(df1)
        wides = df1['wides'].count()
        Balls = balls - wides
        outs = df1['player_dismissed'].loc[df1['player_dismissed'] == name].count()
        runs = df1['runs_off_bat'].sum()
        run = df1['runs_off_bat'].value_counts()

        allPlayersBat[name][phs[i]]['wkt'] = oneDiv(outs, balls)
        allPlayersBat[name][phs[i]]['0'] = zeroDiv(run.get(0, 0) - (wides + outs), Balls) if Balls else phase[phs[i]]['0']
        allPlayersBat[name][phs[i]]['1'] = zeroDiv(run.get(1, 0), Balls)
        allPlayersBat[name][phs[i]]['2'] = zeroDiv(run.get(2, 0), Balls)
        allPlayersBat[name][phs[i]]['3'] = zeroDiv(run.get(3, 0), Balls)
        allPlayersBat[name][phs[i]]['4'] = zeroDiv(run.get(4, 0), Balls)
        allPlayersBat[name][phs[i]]['6'] = zeroDiv(run.get(6, 0), Balls)
        allPlayersBat[name][phs[i]]['runs'] = runs
        allPlayersBat[name][phs[i]]['sr'] = zeroDiv(runs, Balls)
        allPlayersBat[name][phs[i]]['Balls'] = Balls
        allPlayersBat[name][phs[i]]['outs'] = outs
        
        batStats[phs[i]+'_runs'].loc[batStats['Name'] == name] = runs
        batStats[phs[i]+'_avg'].loc[batStats['Name'] == name] = xDiv(runs, outs)
        batStats[phs[i]+'_sr'].loc[batStats['Name'] == name] = zeroDiv(runs, Balls) * 100
        
        df3 = df2.loc[df['wides'].isna()]
        df4 = df3.drop(['venue', 'striker', 'extras', 'innings', 'bowler', 'wides', 'noballs', 'wicket_type', 'player_dismissed'], axis=1)
        runs = {0:0, 1:0, 2:0, 3:0}
        Balls = {0:0, 1:0, 2:0, 3:0}

        for id1 in df4['match_id'].unique():
            df6 = df4.loc[df['match_id'] == id1]
            df5 = df6.sort_values(by = ['ball'])
            if ((df5.iloc[0]['ball'] > st[i]) & (df5.iloc[0]['ball'] < en[i])):
                balls = len(df5)
                l1 = min(((balls//10)+1) if (balls%10) else (balls//10), 4)

                for k in range(l1):
                    runs[k] += df5['runs_off_bat'][k*10 : (k+1)*10].sum()
                    Balls[k] += len(df5[k*10 : (k+1)*10])

                res = [zeroDiv(runs[key], Balls[key]) for key in runs]
                allPlayersBat[name]['SRper10Balls'+phs[i]] = res
        
    df3 = df2.loc[df['wides'].isna()]
    df4 = df3.drop(['venue', 'striker', 'extras', 'innings', 'bowler', 'wides', 'noballs', 'wicket_type', 'player_dismissed'], axis=1)
    runsovr = {0:0, 1:0, 2:0, 3:0}
    Ballsovr = {0:0, 1:0, 2:0, 3:0}
    
    for id1 in df4['match_id'].unique():
        df5 = df4.loc[df['match_id'] == id1]
        balls = len(df5)
        l1 = min(((balls//10)+1) if (balls%10) else (balls//10), 4)

        for l in range(l1):
            runsovr[l] += df5['runs_off_bat'][l*10 : (l+1)*10].sum()
            Ballsovr[l] += len(df5[l*10 : (l+1)*10])
        
        resovr = [zeroDiv(runsovr[key], Ballsovr[key]) for key in runsovr]
        allPlayersBat[name]['SRper10Balls'] = resovr
            

In [91]:
allPlayersBat['MS Dhoni']

{'pp': {'wkt': 0.0256,
  '0': 0.5972,
  '1': 0.2083,
  '2': 0.0278,
  '3': 0.0,
  '4': 0.1389,
  '6': 0.0,
  'runs': 59,
  'sr': 0.8194,
  'Balls': 72,
  'outs': 2},
 'SRper10Ballspp': [0.9054, 1.0909, 1.0857, 1.7209],
 'mid': {'wkt': 0.0213,
  '0': 0.3578,
  '1': 0.4491,
  '2': 0.07,
  '3': 0.0049,
  '4': 0.0624,
  '6': 0.0341,
  'runs': 2389,
  'sr': 1.058,
  'Balls': 2258,
  'outs': 49},
 'SRper10Ballsmid': [1.0773, 1.4571, 1.7143, 2.3145],
 'death': {'wkt': 0.0514,
  '0': 0.2034,
  '1': 0.355,
  '2': 0.128,
  '3': 0.003,
  '4': 0.1442,
  '6': 0.1124,
  'runs': 2530,
  'sr': 1.8713,
  'Balls': 1352,
  'outs': 73},
 'SRper10Ballsdeath': [1.6114, 1.7059, 0, 0],
 'SRper10Balls': [1.1204, 1.4252, 1.6347, 2.1617]}

In [92]:
file = open('D:/programs_demo/T20_sim/extractedData/allPlayersBatData', 'wb')
pickle.dump(allPlayersBat, file)
file.close()

In [93]:
allPlayersBowl = {}

bowlStats = pd.DataFrame(columns = ['Name', 'pp_wkts', 'pp_balls', 'pp_bowl_sr', 'pp_eco', 'pp_bowl_avg', 'mid_wkts', 'mid_balls', 'mid_bowl_sr', 'mid_eco', 'mid_bowl_avg', 'death_wkts', 'death_balls', 'death_bowl_sr', 'death_bowl_avg', 'death_eco'])

phs = ['pp', 'mid', 'death']
st = [0, 6, 16]
en = [6, 16, 20]

bowlStats['Name'] = df['bowler'].unique()

for name in df['bowler'].unique():
    allPlayersBowl[name] = {}
    df2 = df.loc[df['bowler'] == name]
    
    for i in range(3):
        allPlayersBowl[name][phs[i]] = {}
        df1 = df2.loc[(df2['ball'] > st[i]) & (df2['ball'] < en[i])]
        balls = len(df1)
        wides = df1['wides'].count()
        Balls = balls - wides
        wkts = df1['wicket_type'].loc[df1['wicket_type'] != 'run out'].count()
        runs_given = df1['runs_off_bat'].sum() + df1['wides'].sum() + df1['noballs'].sum()
        run = df1['runs_off_bat'].value_counts()

        allPlayersBowl[name][phs[i]]['wkt'] = zeroDiv(wkts, balls)
        allPlayersBowl[name][phs[i]]['wide'] = zeroDiv(wides, balls) if Balls else phase[phs[i]]['wide']
        allPlayersBowl[name][phs[i]]['noball'] = zeroDiv(df1['noballs'].count(), balls) if Balls else phase[phs[i]]['noball']
        allPlayersBowl[name][phs[i]]['0'] = zeroDiv(run.get(0, 0) - (wides + wkts), Balls) if Balls else phase[phs[i]]['0']
        allPlayersBowl[name][phs[i]]['1'] = oneDiv(run.get(1, 0), Balls)
        allPlayersBowl[name][phs[i]]['2'] = oneDiv(run.get(2, 0), Balls)
        allPlayersBowl[name][phs[i]]['3'] = zeroDiv(run.get(3, 0), Balls) if Balls else phase[phs[i]]['3']
        allPlayersBowl[name][phs[i]]['4'] = oneDiv(run.get(4, 0), Balls)
        allPlayersBowl[name][phs[i]]['6'] = oneDiv(run.get(6, 0), Balls)
        allPlayersBowl[name][phs[i]]['runs_given'] = runs_given
        allPlayersBowl[name][phs[i]]['eco'] = round((zeroDiv(runs_given, Balls) * 6), 4)
        allPlayersBowl[name][phs[i]]['balls'] = balls
        allPlayersBowl[name][phs[i]]['wkts'] = wkts
        
        bowlStats[phs[i]+'_balls'].loc[bowlStats['Name'] == name] = balls
        bowlStats[phs[i]+'_wkts'].loc[bowlStats['Name'] == name] = wkts
        bowlStats[phs[i]+'_eco'].loc[bowlStats['Name'] == name] = allPlayersBowl[name][phs[i]]['eco']
        bowlStats[phs[i]+'_bowl_avg'].loc[bowlStats['Name'] == name] = zeroDiv(runs_given, wkts)
        bowlStats[phs[i]+'_bowl_sr'].loc[bowlStats['Name'] == name] = zeroDiv(balls, wkts)
        

In [94]:
file = open('D:/programs_demo/T20_sim/extractedData/allPlayersBowlData', 'wb')
pickle.dump(allPlayersBowl, file)
file.close()

In [95]:
for i in range(2):
    bowlStats[phs[i]+'_bowl_avg'].mask(bowlStats[phs[i]+'_wkts'] == 0, np.NAN, inplace = True)
    bowlStats[phs[i]+'_bowl_sr'].mask(bowlStats[phs[i]+'_wkts'] == 0, np.NAN, inplace = True)

In [96]:
for i in range(2):
    bowlStats[phs[i]+'_eco'].mask(bowlStats[phs[i]+'_balls'] == 0, np.NAN, inplace = True)

In [97]:
stats = batStats.merge(bowlStats, on = 'Name', how='outer')

In [98]:
stats.to_csv('D:/programs_demo/T20_sim/extractedData/Stats-All-Players.csv', na_rep = "NA")

In [99]:
expStats = stats.convert_dtypes()

In [100]:
import math

In [101]:
worstbat = {}
worstbowl = {}

for i in range(3):
    worstbat[phs[i]] = {}
    worstbat[phs[i]]['wkt'] = 4 * phase[phs[i]]['wkt'] 
    worstbat[phs[i]]['0'] = phase[phs[i]]['0']
    worstbat[phs[i]]['1'] = phase[phs[i]]['1']
    worstbat[phs[i]]['2'] = phase[phs[i]]['2']
    worstbat[phs[i]]['3'] = phase[phs[i]]['3']
    worstbat[phs[i]]['4'] = phase[phs[i]]['4'] / 4
    worstbat[phs[i]]['6'] = phase[phs[i]]['6'] / 4
    worstbat[phs[i]]['sr'] = round((1*worstbat[phs[i]]['1'] + 2*worstbat[phs[i]]['2'] + 3*worstbat[phs[i]]['3'] + 4*worstbat[phs[i]]['4'] + 6*worstbat[phs[i]]['6']) / (worstbat[phs[i]]['wkt'] + worstbat[phs[i]]['0'] + worstbat[phs[i]]['1'] + worstbat[phs[i]]['2'] + worstbat[phs[i]]['3'] + worstbat[phs[i]]['4'] + worstbat[phs[i]]['6']), 4)
    
    worstbowl[phs[i]] = {}
    worstbowl[phs[i]]['wkt'] =  round(phase[phs[i]]['wkt'] / 5, 4)
    worstbowl[phs[i]]['noball'] = phase[phs[i]]['noball']
    worstbowl[phs[i]]['wide'] = phase[phs[i]]['wide']
    worstbowl[phs[i]]['0'] = phase[phs[i]]['0']
    worstbowl[phs[i]]['1'] = 1
    worstbowl[phs[i]]['2'] = 1
    worstbowl[phs[i]]['3'] = phase[phs[i]]['3']
    worstbowl[phs[i]]['4'] = 1
    worstbowl[phs[i]]['6'] = 1
    
    worstbowl[phs[i]]['eco'] = round(6 * ((worstbowl[phs[i]]['wide'] + worstbowl[phs[i]]['noball'] + 1*worstbowl[phs[i]]['1'] + 2*worstbowl[phs[i]]['2'] + 3*worstbowl[phs[i]]['3'] + 4*worstbowl[phs[i]]['4'] + 6*worstbowl[phs[i]]['6']) / (worstbowl[phs[i]]['wkt'] + worstbowl[phs[i]]['0'] + worstbowl[phs[i]]['1'] + worstbowl[phs[i]]['2'] + worstbowl[phs[i]]['3'] + worstbowl[phs[i]]['4'] + worstbowl[phs[i]]['6'])), 4)
    
    worstbat['SRper10Balls'+phs[i]] = [0, 0, 0, 0]

worstbat['SRper10Balls'] = [0, 0, 0, 0]

In [102]:
playerList = list(set(list(allPlayersBat.keys()) + list(allPlayersBowl.keys())))

adjustedDataAll = {}

batCutoff = [500, 600, 500]
bowlCutoff = [300, 300, 300]

for i in playerList:
    adjustedDataAll[i] = {}
    adjustedDataAll[i]['name'] = i
    
    if (allPlayersBat.get(i, -1) == -1):
        adjustedDataAll[i]['bat'] = worstbat.copy()
    
    else:
        adjustedDataAll[i]['bat'] = {}
        
        for j in range(3):
            if (allPlayersBat[i][phs[j]]['runs'] == 0):
                adjustedDataAll[i]['bat'][phs[j]] = worstbat[phs[j]].copy()
                adjustedDataAll[i]['bat']['SRper10Balls'+phs[j]] = worstbat['SRper10Balls'+phs[j]]
                adjustedDataAll[i]['bat']['SRper10Balls'] = worstbat['SRper10Balls']
            else:
                a = np.clip(round(math.log10(allPlayersBat[i][phs[j]]['runs']/ batCutoff[j]), 4), -1, 0)
                
                adjustedDataAll[i]['bat'][phs[j]] = {}
                
                adjustedDataAll[i]['bat'][phs[j]]['wkt'] = round(allPlayersBat[i][phs[j]]['wkt'] * (1 + a) - (a * worstbat[phs[j]]['wkt']), 4)
                
                adjustedDataAll[i]['bat'][phs[j]]['0'] = round(allPlayersBat[i][phs[j]]['0'] * (1 + a) - (a * worstbat[phs[j]]['0']), 4)
                
                adjustedDataAll[i]['bat'][phs[j]]['1'] = round(allPlayersBat[i][phs[j]]['1'] * (1 + a) - (a * worstbat[phs[j]]['1']), 4)
                
                adjustedDataAll[i]['bat'][phs[j]]['2'] = round(allPlayersBat[i][phs[j]]['2'] * (1 + a) - (a * worstbat[phs[j]]['2']), 4)
                
                adjustedDataAll[i]['bat'][phs[j]]['3'] = round(allPlayersBat[i][phs[j]]['3'] * (1 + a) - (a * worstbat[phs[j]]['3']), 4)
                
                adjustedDataAll[i]['bat'][phs[j]]['4'] = round(allPlayersBat[i][phs[j]]['4'] * (1 + a) - (a * worstbat[phs[j]]['4']), 4)
                
                adjustedDataAll[i]['bat'][phs[j]]['6'] = round(allPlayersBat[i][phs[j]]['6'] * (1 + a) - (a * worstbat[phs[j]]['6']), 4)
                
                adjustedDataAll[i]['bat'][phs[j]]['sr'] = round((1*adjustedDataAll[i]['bat'][phs[j]]['1'] + 2*adjustedDataAll[i]['bat'][phs[j]]['2'] + 3*adjustedDataAll[i]['bat'][phs[j]]['3'] + 4*adjustedDataAll[i]['bat'][phs[j]]['4'] + 6*adjustedDataAll[i]['bat'][phs[j]]['6']) / (adjustedDataAll[i]['bat'][phs[j]]['wkt'] + adjustedDataAll[i]['bat'][phs[j]]['0'] + adjustedDataAll[i]['bat'][phs[j]]['1'] + adjustedDataAll[i]['bat'][phs[j]]['2'] + adjustedDataAll[i]['bat'][phs[j]]['3'] + adjustedDataAll[i]['bat'][phs[j]]['4'] + adjustedDataAll[i]['bat'][phs[j]]['6']), 4)
                
                try:
                    adjustedDataAll[i]['bat']['SRper10Balls'+phs[j]] = allPlayersBat[i]['SRper10Balls'+phs[j]]
                except KeyError:
                    adjustedDataAll[i]['bat']['SRper10Balls'+phs[j]] = worstbat['SRper10Balls'+phs[j]]
                
        adjustedDataAll[i]['bat']['SRper10Balls'] = allPlayersBat[i]['SRper10Balls']
        

In [103]:
for i in playerList:
    
    if (allPlayersBowl.get(i, -1) == -1):
        adjustedDataAll[i]['bowl'] = worstbowl.copy()
    
    else:
        adjustedDataAll[i]['bowl'] = {}
        
        for j in range(3):
            if (allPlayersBowl[i][phs[j]]['balls'] == 0):
                adjustedDataAll[i]['bowl'][phs[j]] = worstbowl[phs[j]].copy()
            else:
                a = np.clip(round(math.log10(allPlayersBowl[i][phs[j]]['balls']/ bowlCutoff[j]), 4), -1, 0)
                
                adjustedDataAll[i]['bowl'][phs[j]] = {}
                
                adjustedDataAll[i]['bowl'][phs[j]]['wide'] = round(allPlayersBowl[i][phs[j]]['wide'] * (1 + a) - (a * worstbowl[phs[j]]['wide']), 4)
                
                adjustedDataAll[i]['bowl'][phs[j]]['noball'] = round(allPlayersBowl[i][phs[j]]['noball'] * (1 + a) - (a * worstbowl[phs[j]]['noball']), 4)
                
                adjustedDataAll[i]['bowl'][phs[j]]['wkt'] = round(allPlayersBowl[i][phs[j]]['wkt'] * (1 + a) - (a * worstbowl[phs[j]]['wkt']), 4)
                
                adjustedDataAll[i]['bowl'][phs[j]]['0'] = round(allPlayersBowl[i][phs[j]]['0'] * (1 + a) - (a * worstbowl[phs[j]]['0']), 4)
                
                adjustedDataAll[i]['bowl'][phs[j]]['1'] = round(allPlayersBowl[i][phs[j]]['1'] * (1 + a) - (a * worstbowl[phs[j]]['1']), 4)
                
                adjustedDataAll[i]['bowl'][phs[j]]['2'] = round(allPlayersBowl[i][phs[j]]['2'] * (1 + a) - (a * worstbowl[phs[j]]['2']), 4)
                
                adjustedDataAll[i]['bowl'][phs[j]]['3'] = round(allPlayersBowl[i][phs[j]]['3'] * (1 + a) - (a * worstbowl[phs[j]]['3']), 4)
                
                adjustedDataAll[i]['bowl'][phs[j]]['4'] = round(allPlayersBowl[i][phs[j]]['4'] * (1 + a) - (a * worstbowl[phs[j]]['4']), 4)
                
                adjustedDataAll[i]['bowl'][phs[j]]['6'] = round(allPlayersBowl[i][phs[j]]['6'] * (1 + a) - (a * worstbowl[phs[j]]['6']), 4)
                
                adjustedDataAll[i]['bowl'][phs[j]]['eco'] = round(6 * ((adjustedDataAll[i]['bowl'][phs[j]]['wide'] + adjustedDataAll[i]['bowl'][phs[j]]['noball'] + 1*adjustedDataAll[i]['bowl'][phs[j]]['1'] + 2*adjustedDataAll[i]['bowl'][phs[j]]['2'] + 3*adjustedDataAll[i]['bowl'][phs[j]]['3'] + 4*adjustedDataAll[i]['bowl'][phs[j]]['4'] + 6*adjustedDataAll[i]['bowl'][phs[j]]['6']) / (adjustedDataAll[i]['bowl'][phs[j]]['wkt'] + adjustedDataAll[i]['bowl'][phs[j]]['0'] + adjustedDataAll[i]['bowl'][phs[j]]['1'] + adjustedDataAll[i]['bowl'][phs[j]]['2'] + adjustedDataAll[i]['bowl'][phs[j]]['3'] + adjustedDataAll[i]['bowl'][phs[j]]['4'] + adjustedDataAll[i]['bowl'][phs[j]]['6'])), 4)

In [104]:
bat = expStats.loc[stats['death_balls'] > 200].sort_values('death_eco', ascending = True).reset_index()
bat.loc[0:40, ['Name','death_wkts', 'death_balls', 'death_bowl_avg', 'death_bowl_sr', 'death_eco']]

,Name,death_wkts,death_balls,death_bowl_avg,death_bowl_sr,death_eco
0,Rashid Khan,14,219,18.5,15.6429,7.1946
1,DE Bollinger,21,216,12.5238,10.2857,7.4784
2,SL Malinga,90,965,13.2889,10.7222,7.7664
3,Mohammad Sami,23,243,13.3478,10.5652,7.7724
4,Imran Tahir,30,255,11.0,8.5,7.8882
5,A Nortje,15,207,17.4667,13.8,7.9392
6,SP Narine,58,748,16.9655,12.8966,8.0436
7,Wahab Riaz,50,594,15.12,11.88,8.0568
8,Shaheen Shah Afridi,29,427,19.0345,14.7241,8.0778
9,KK Cooper,20,281,18.7,14.05,8.2806


In [105]:
file = open('D:/programs_demo/T20_sim/extractedData/allDataAdjusted', 'wb')
pickle.dump(adjustedDataAll, file)
file.close()

In [106]:
import json
with open('D:/programs_demo/T20_sim/extractedData/adjustedDataAll.json', 'w') as fp:
    json.dump(adjustedDataAll, fp)

In [ ]:
import pandas
roles = pd.read_csv('C:/users/shara/Downloads/roles.csv')
role_df = roles.dropna()

for i in range(len(role_df)):
    playersData[role_df.iloc[i]['Name']]['role'] = role_df.iloc[i]["Role"]

import pickle
file = open('D:/programs_demo/T20_sim/extractedData/allDataAdjusted', 'wb')
pickle.dump(playersData, file)
file.close()

import json
with open('D:/programs_demo/T20_sim/extractedData/adjustedDataAll.json', 'w') as fp:
    json.dump(playersData, fp)